In [1]:
import os
import json
import logging
import pickle
import shutil
from datetime import datetime
import pandas as pd
import numpy as np
from pathlib import Path
from gensim.corpora import Dictionary
from gensim.models import LdaModel
from scipy.stats import entropy
from itertools import combinations
from typing import List, Dict, Tuple, Set, Any
from tqdm import tqdm
import multiprocessing as mp
from functools import partial
from multiprocessing import Pool

# 设置工作目录和路径
BASE_DIR = "/Users/jessie/Documents/Projects/Cusanus_Topic_Modeling"
os.chdir(BASE_DIR)



In [2]:
def setup_logging() -> str:
    """设置日志"""
    log_dir = Path('experiments/lda/logs')
    log_dir.mkdir(parents=True, exist_ok=True)
    
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    log_file = log_dir / f'experiment_{timestamp}.log'
    
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler(log_file),
            logging.StreamHandler()
        ]
    )
    return log_file


In [3]:
def load_documents(directory: Path) -> List[List[str]]:
    """加载文档，只在文档级别处理"""
    documents = []
    
    for file_path in directory.glob('*.txt'):
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read().strip()
            
            # 将整个文档内容合并为一个词列表
            words = []
            # 将所有内容合并，不按段落分割
            text = ' '.join([p.split(':', 1)[-1].strip() 
                           for p in content.split('Paragraph') 
                           if p.strip()])
            
            words = text.split()
            if words:  # 只添加非空文档
                documents.append(words)
                logging.info(f"成功加载文档: {file_path}")
                
        except Exception as e:
            logging.error(f"加载文档 {file_path} 时发生错误: {str(e)}")
    
    logging.info(f"总共加载了 {len(documents)} 个文档")
    return documents

In [4]:
def calculate_word_cooccurrences(texts: List[List[str]], window_size: int = 10) -> Dict[Tuple[str, str], int]:
    """计算词对在文档中的共现次数"""
    cooccurrences = {}
    
    for text in texts:
        for i in range(len(text)):
            # 在窗口大小内查找共现词
            window = text[max(0, i-window_size):min(len(text), i+window_size+1)]
            for j, word1 in enumerate(window):
                for word2 in window[j+1:]:
                    if word1 < word2:  # 确保词对顺序一致
                        pair = (word1, word2)
                    else:
                        pair = (word2, word1)
                    cooccurrences[pair] = cooccurrences.get(pair, 0) + 1
                    
    return cooccurrences

def calculate_weighted_npmi(model: LdaModel, texts: List[List[str]], 
                          dictionary: Dictionary, top_n: int = 10, 
                          window_size: int = 10, eps: float = 1e-12) -> float:
    """计算加权NPMI分数"""
    try:
        # 获取词共现统计
        cooccurrences = calculate_word_cooccurrences(texts, window_size)
        
        # 计算单词频率
        word_freq = {}
        total_windows = 0
        for text in texts:
            total_windows += max(1, len(text) - window_size + 1)
            for word in text:
                word_freq[word] = word_freq.get(word, 0) + 1
        
        weighted_npmi_sum = 0
        weight_sum = 0
        
        # 对每个主题计算加权NPMI
        for topic_id in range(model.num_topics):
            topic_words = [word for word, _ in model.show_topic(topic_id, topn=top_n)]
            
            # 计算词对的NPMI
            for i, word1 in enumerate(topic_words):
                for word2 in topic_words[i+1:]:
                    if word1 < word2:
                        pair = (word1, word2)
                    else:
                        pair = (word2, word1)
                    
                    count = cooccurrences.get(pair, 0)
                    if count > 0:
                        # 计算联合概率和边缘概率
                        p_xy = (count + eps) / total_windows
                        p_x = (word_freq.get(word1, 0) + eps) / total_windows
                        p_y = (word_freq.get(word2, 0) + eps) / total_windows
                        
                        # 计算PMI和NPMI
                        pmi = np.log(p_xy / (p_x * p_y))
                        npmi = pmi / (-np.log(p_xy))
                        
                        # 加权累加
                        weighted_npmi_sum += npmi * count
                        weight_sum += count
        
        # 标准化到[0,1]区间
        if weight_sum > 0:
            weighted_npmi = (weighted_npmi_sum / weight_sum + 1) / 2
        else:
            weighted_npmi = 0
            
        return max(0.0, min(1.0, weighted_npmi))
    
    except Exception as e:
        logging.error(f"计算加权NPMI时发生错误: {str(e)}")
        return 0.0

def calculate_improved_diversity(model: LdaModel, beta: float = 0.5, 
                               top_n: int = 10, eps: float = 1e-12) -> float:
    """计算改进的主题多样性分数"""
    try:
        K = model.num_topics
        total_score = 0
        num_pairs = 0
        
        # 获取所有主题的词分布
        topic_distributions = []
        topic_words = []
        for topic_id in range(K):
            topic_dist = dict(model.show_topic(topic_id, topn=top_n))
            topic_distributions.append([v for _, v in sorted(topic_dist.items())])
            topic_words.append(set(topic_dist.keys()))
        
        # 计算所有主题对的多样性
        for i, j in combinations(range(K), 2):
            # 计算JSD
            M = [(p1 + p2) / 2 for p1, p2 in zip(topic_distributions[i], topic_distributions[j])]
            jsd = (entropy(topic_distributions[i], M) + entropy(topic_distributions[j], M)) / 2
            
            # 计算词重叠度
            intersection = len(topic_words[i] & topic_words[j])
            min_size = min(len(topic_words[i]), len(topic_words[j]))
            overlap = 1 - (intersection / min_size if min_size > 0 else 0)
            
            # 组合得分
            pair_score = beta * jsd + (1 - beta) * overlap
            total_score += pair_score
            num_pairs += 1
        
        # 计算平均多样性分数
        diversity = total_score / num_pairs if num_pairs > 0 else 0
        return max(0.0, min(1.0, diversity))
    
    except Exception as e:
        logging.error(f"计算改进的多样性分数时发生错误: {str(e)}")
        return 0.0

def calculate_optimal_score(model: LdaModel, texts: List[List[str]], 
                          dictionary: Dictionary, alpha: float = 0.5, 
                          beta: float = 0.5, top_n: int = 10) -> float:
    """计算最优分数"""
    try:
        # 计算加权NPMI
        weighted_npmi = calculate_weighted_npmi(model, texts, dictionary, top_n=top_n)
        
        # 计算改进的多样性
        improved_diversity = calculate_improved_diversity(model, beta=beta, top_n=top_n)
        
        # 计算最终得分
        optimal_score = alpha * weighted_npmi + (1 - alpha) * improved_diversity
        return optimal_score
    
    except Exception as e:
        logging.error(f"计算最优分数时发生错误: {str(e)}")
        return 0.0


In [5]:
def evaluate_single_parameter(params: tuple, train_docs: List[List[str]], n_passes: int = 100) -> dict:
    """评估单个参数组合"""
    min_freq, max_freq, num_topics, alpha, eta = params
    
    try:
        # 创建词典
        dictionary = Dictionary(train_docs)
        dictionary.filter_extremes(no_below=min_freq, no_above=max_freq/len(train_docs))
        
        # 创建语料库
        corpus = [dictionary.doc2bow(doc) for doc in train_docs]
        
        # 训练LDA模型
        model = LdaModel(
            corpus=corpus,
            id2word=dictionary,
            num_topics=num_topics,
            alpha=alpha,
            eta=eta,
            iterations=n_passes,  # 使用传入的n_passes参数
            random_state=42
        )
        
        # 计算评分（保持原有的评估方式）
        score = calculate_optimal_score(
            model=model,
            texts=train_docs,
            dictionary=dictionary,
            alpha=0.5  # 用于NPMI和多样性的权重
        )
        
        return {
            'min_freq': min_freq,
            'max_freq': max_freq,
            'num_topics': num_topics,
            'alpha': alpha,
            'eta': eta,
            'optimal_score': score,
            'model': model,
            'dictionary': dictionary
        }
        
    except Exception as e:
        logging.error(f"参数组合评估失败: {str(e)}")
        return None

def evaluate_parameters_parallel(
    grid_params: dict, 
    train_docs: List[List[str]], 
    experiment_dir: Path, 
    use_multiprocessing: bool = True,
    n_passes: int = 100,  # 添加n_passes参数
    **kwargs  # 添加kwargs来处理其他可能的参数
) -> pd.DataFrame:
    """并行评估所有参数组合"""
    # 获取参数范围
    min_freqs = grid_params.get('min_freqs', [2, 3, 4])
    max_freqs = grid_params.get('max_freqs', [200, 800, 1400, 2000])
    num_topics = grid_params.get('num_topics', [10])
    alpha_range = grid_params.get('alpha_range', ['symmetric', 0.1, 0.3, 0.5, 0.7])
    eta_range = grid_params.get('eta_range', ['symmetric', 0.1, 0.3, 0.5, 0.7])
    
    # 生成参数组合
    params_combinations = [
        (min_freq, max_freq, num_topic, alpha, eta)
        for min_freq in min_freqs
        for max_freq in max_freqs
        for num_topic in num_topics
        for alpha in alpha_range
        for eta in eta_range
    ]
    
    eval_func = partial(
        evaluate_single_parameter, 
        train_docs=train_docs,
        n_passes=n_passes  # 传递n_passes参数
    )
    
    if use_multiprocessing:
        with Pool() as pool:
            results = list(tqdm(
                pool.imap(eval_func, params_combinations), 
                total=len(params_combinations)
            ))
    else:
        results = list(tqdm(
            map(eval_func, params_combinations), 
            total=len(params_combinations)
        ))
    
    # 过滤掉失败的结果并转换为DataFrame
    valid_results = [r for r in results if r is not None]
    results_df = pd.DataFrame(valid_results)
    
    # 按optimal_score排序并保存
    results_df = results_df.sort_values('optimal_score', ascending=False)
    results_df.to_csv(experiment_dir / 'parameter_evaluation_results.csv', index=False)
    
    return results_df

def save_evaluation_results(results_df: pd.DataFrame, model: LdaModel, 
                          prep_type: str, experiment_dir: Path) -> Dict:
    """保存评估结果"""
    results_dir = experiment_dir / prep_type / 'results'
    results_dir.mkdir(parents=True, exist_ok=True)
    
    try:
        # 保存基础评估结果
        results_df.to_csv(results_dir / 'evaluation_results.csv', index=False)
        
        # 保存主题词分布
        topic_words = {}
        for topic_id in range(model.num_topics):
            words = [word for word, prob in model.show_topic(topic_id, topn=20)]
            topic_words[f'Topic_{topic_id}'] = words
            
        with open(results_dir / 'topic_words.json', 'w', encoding='utf-8') as f:
            json.dump(topic_words, f, indent=2, ensure_ascii=False)
            
        # 保存模型
        model.save(str(results_dir / 'best_model.lda'))
        
        # 保存详细结果
        detailed_results = {
            'model_params': {
                'num_topics': model.num_topics,
                'alpha': model.alpha,
                'eta': model.eta,
                'iterations': model.iterations
            },
            'evaluation_metrics': {
                'optimal_score': float(results_df['optimal_score'].max()),
                'avg_score': float(results_df['optimal_score'].mean()),
                'std_score': float(results_df['optimal_score'].std())
            },
            'topic_coherence': {
                'npmi_score': float(results_df['npmi_score'].max()),
                'diversity_score': float(results_df['diversity_score'].max())
            }
        }
        
        with open(results_dir / 'detailed_results.json', 'w', encoding='utf-8') as f:
            json.dump(detailed_results, f, indent=2)
            
        return detailed_results
        
    except Exception as e:
        logging.error(f"保存评估结果时发生错误: {str(e)}")
        return None

In [6]:
def create_summary_table(results_dict: Dict, n_topics: int = 10) -> pd.DataFrame:
    """创建实验结果汇总表格"""
    summary_data = []
    
    for prep_type, results_path in results_dict.items():
        try:
            # 加载final_results.json
            with open(results_path, 'r', encoding='utf-8') as f:
                results = json.load(f)
            
            # 处理top 5结果
            for idx, params in enumerate(results['top_5_params'], 1):
                # 获取主题词（每个主题前10个词）
                model_path = os.path.join(os.path.dirname(results_path), f'model_{idx}.lda')
                if os.path.exists(model_path):
                    model = LdaModel.load(model_path)
                    # 为每个主题创建关键词列表
                    topics_keywords = {}
                    for topic_id in range(n_topics):
                        words = [word for word, _ in model.show_topic(topic_id, topn=10)]
                        topics_keywords[f'Topic_{topic_id+1}'] = ', '.join(words)
                    
                    # 创建基本信息
                    entry = {
                        'Exp. ID': f"{prep_type[:3]}-{idx}",
                        'Lemmatization Method': prep_type,
                        'Threshold': f"{params['min_freq']}-{params['max_freq']}",
                        'alpha': params['alpha'],
                        'eta': 'auto',
                        'n_topics': n_topics,
                        'n_passes': params['n_passes'],
                        'optimal score': params['test_score']
                    }
                    
                    # 添加每个主题的关键词
                    entry.update(topics_keywords)
                    summary_data.append(entry)
                
        except Exception as e:
            logging.error(f"处理 {prep_type} 结果时发生错误: {str(e)}")
    
    # 创建DataFrame
    summary_df = pd.DataFrame(summary_data)
    
    # 保存为CSV
    os.makedirs('experiments/lda/summaries', exist_ok=True)
    summary_df.to_csv('experiments/lda/summaries/experiment_summary.csv', index=False)
    
    # Excel格式（带格式）
    with pd.ExcelWriter('experiments/lda/summaries/experiment_summary.xlsx', engine='xlsxwriter') as writer:
        summary_df.to_excel(writer, index=False, sheet_name='Summary')
        
        workbook = writer.book
        worksheet = writer.sheets['Summary']
        
        # 设置列宽
        base_columns = {
            'A': 10,  # Exp. ID
            'B': 20,  # Lemmatization Method
            'C': 15,  # Threshold
            'D': 10,  # alpha
            'E': 10,  # eta
            'F': 10,  # n_topics
            'G': 10,  # n_passes
            'H': 15,  # optimal score
        }
        
        # 为每个主题的关键词添加列宽
        topic_columns = {chr(ord('I') + i): 100 for i in range(n_topics)}
        column_widths = {**base_columns, **topic_columns}
        
        for col, width in column_widths.items():
            worksheet.set_column(f'{col}:{col}', width)
        
        # 添加表格格式
        header_format = workbook.add_format({
            'bold': True,
            'bg_color': '#D9E1F2',
            'border': 1,
            'text_wrap': True,
            'align': 'center',
            'valign': 'vcenter'
        })
        
        # 应用表头格式
        for col_num, value in enumerate(summary_df.columns.values):
            worksheet.write(0, col_num, value, header_format)
        
        # 添加数据单元格格式
        data_format = workbook.add_format({
            'text_wrap': True,
            'valign': 'top',
            'border': 1
        })
        
        # 应用数据格式
        for row in range(1, len(summary_df) + 1):
            for col in range(len(summary_df.columns)):
                worksheet.write(row, col, summary_df.iloc[row-1, col], data_format)
    
    # Markdown格式
    with open('experiments/lda/summaries/experiment_summary.md', 'w', encoding='utf-8') as f:
        f.write(summary_df.to_markdown(index=False))
    
    return summary_df

In [ ]:
def run_experiment():
    """运行完整实验流程"""
    # 设置日志
    log_file = setup_logging()
    logging.info("开始LDA实验流程")
    
    # 实验参数
    grid_params = {
        'min_freqs': [2, 3, 4, 5],
        'max_freqs': [200, 500, 800, 1000, 1500],
        'num_topics': [10],
        'alpha_range': [0.3, 0.5, 0.7],
        'eta_range': ['symmetric']
    }
    
    # 其他训练参数
    training_params = {
        'n_passes': 100,
        'random_state': 42
    }
    
    try:
        experiment_dir = Path(BASE_DIR) / 'experiments/lda'
        results_paths = {}
        
        # 处理两种预处理方法
        for prep_type in ['spacy', 'cusanus']:
            logging.info(f"\n{'='*50}")
            logging.info(f"开始处理 {prep_type} 预处理数据")
            logging.info(f"{'='*50}")
            
            try:
                # 加载训练集和测试集
                train_dir = experiment_dir / prep_type / 'train_set'
                test_dir = experiment_dir / prep_type / 'test_set'
                
                train_docs = load_documents(train_dir)
                test_docs = load_documents(test_dir)
                
                logging.info(f"加载了 {len(train_docs)} 个训练文档和 {len(test_docs)} 个测试文档")
                
                # 创建当前预处理方法的实验目录
                prep_experiment_dir = experiment_dir / prep_type
                prep_experiment_dir.mkdir(parents=True, exist_ok=True)
                
                # 进行参数评估
                logging.info("\n开始参数网格搜索...")
                try:
                    results_df = evaluate_parameters_parallel(
                        grid_params=grid_params,
                        train_docs=train_docs,
                        experiment_dir=prep_experiment_dir,
                        use_multiprocessing=True,
                        **training_params
                    )
                except Exception as e:
                    logging.error(f"参数评估过程中发生错误: {str(e)}")
                    raise
                
                # 创建结果目录
                results_dir = prep_experiment_dir / 'results'
                results_dir.mkdir(parents=True, exist_ok=True)
                
                # 保存训练结果
                results_df.to_csv(results_dir / 'train_results.csv', index=False)
                logging.info(f"保存训练结果到: {results_dir / 'train_results.csv'}")
                
                # 获取top 10参数进行测试
                logging.info("\n开始测试阶段...")
                top_10_params = results_df.nlargest(10, 'optimal_score')
                test_results = []
                
                for idx, row in enumerate(top_10_params.iterrows(), 1):
                    _, params_dict = row
                    logging.info(f"测试第 {idx}/10 组参数...")
                    
                    model = params_dict['model']
                    dictionary = params_dict['dictionary']
                    
                    # 计算测试集分数
                    test_score = calculate_optimal_score(
                        model=model,
                        texts=test_docs,
                        dictionary=dictionary,
                        alpha=params_dict['alpha']
                    )
                    
                    test_results.append({
                        **params_dict.to_dict(),
                        'test_score': test_score
                    })
                
                # 保存测试结果
                test_results_df = pd.DataFrame(test_results)
                test_results_df.to_csv(results_dir / 'test_results.csv', index=False)
                
                # 获取最终的top 5结果
                top_5_results = test_results_df.nlargest(5, 'test_score')
                
                # 保存详细结果
                results_paths[prep_type] = results_dir / 'final_results.json'
                with open(results_paths[prep_type], 'w', encoding='utf-8') as f:
                    json.dump({
                        'top_5_params': top_5_results.to_dict('records'),
                        'best_score': float(top_5_results['test_score'].max()),
                        'average_score': float(top_5_results['test_score'].mean()),
                        'std_score': float(top_5_results['test_score'].std()),
                        'experiment_params': {
                            **grid_params,
                            **training_params
                        }
                    }, f, indent=2)
                
                # 保存top 5模型和词典
                for idx, (_, params_dict) in enumerate(top_5_results.iterrows(), 1):
                    model = params_dict['model']
                    dictionary = params_dict['dictionary']
                    
                    # 保存模型
                    model_path = results_dir / f'model_{idx}.lda'
                    model.save(str(model_path))
                    
                    # 保存词典
                    dict_path = results_dir / f'dictionary_{idx}.dict'
                    dictionary.save(str(dict_path))
                
                logging.info(f"\n{prep_type} 处理完成")
                
            except Exception as e:
                logging.error(f"{prep_type} 处理过程中发生错误: {str(e)}")
                continue
        
        # 创建汇总表格
        logging.info("\n创建实验汇总表格...")
        summary_df = create_summary_table(
            results_paths, 
            n_topics=grid_params['num_topics'][0]
        )
        
        logging.info("\n实验完成！")
        logging.info(f"详细日志已保存至: {log_file}")
        logging.info(f"汇总表格已保存至: experiments/lda/summaries/")
        
        # 打印最佳结果
        for prep_type in results_paths:
            with open(results_paths[prep_type], 'r') as f:
                results = json.load(f)
            logging.info(f"\n{prep_type} 最佳结果:")
            logging.info(f"最佳分数: {results['best_score']:.4f}")
            logging.info(f"平均分数: {results['average_score']:.4f}")
            logging.info(f"标准差: {results.get('std_score', 'N/A')}")
        
    except Exception as e:
        logging.error(f"实验过程中发生错误: {str(e)}")
        raise

if __name__ == "__main__":
    run_experiment()